In [11]:
import sys
import numpy as np
import json
import transformers
from transformers import CamembertForMaskedLM
from transformers import CamembertTokenizer as token
from transformers import CamembertForSequenceClassification,TFCamembertForSequenceClassification
from transformers import AutoModel, AutoTokenizer, BertTokenizer
from transformers import CamembertConfig,CamembertTokenizer,CamembertModel
import tokenizers
import pandas as pd
sys.path.append('../')
import lib
import os
import torch

# 1 Load Data

In [6]:
!ls ../data/

Data_cleaned.csv        data                    lequotidien.json
Macky_Sall_twitter.json data.zip                lesoleil.json


In [7]:
df = pd.read_csv("../data/Data_cleaned.csv")

# 2 Load Tokenizer And Camembert Model

In [17]:
# Store the model we want to use
MODEL_NAME = "camembert-base"

# 2 POSSIBILITIES TO LOAD CAMEMBERT

#1st
config = CamembertConfig.from_pretrained(MODEL_NAME)    # Download configuration from S3 and cache.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_config(config) 


#2nd
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertForSequenceClassification.from_pretrained("camembert-base")
#or CamembertModel,CamembertForMaskedLM,CamembertForSequenceClassification,CamembertForMultipleChoice
camembert.eval() 

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [9]:
## Tokens comes from a process that splits the input into sub-entities with interesting linguistic properties. 
#tokens = tokenizer.tokenize("ceci est un exemple !")
#print("Tokens: {}".format(tokens))
#
## This is not sufficient for the model, as it requires integers as input, 
## not a problem, let's convert tokens to ids.
#tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
#print("Tokens id: {}".format(tokens_ids))
#
## Add the required special tokens
#tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)
#
## We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
#tokens_pt = torch.tensor([tokens_ids])
#print("Tokens PyTorch: {}".format(tokens_pt))
#
## Now we're ready to go through BERT with out input
#outputs, pooled = model(tokens_pt)
#print("Token wise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))

# 3 Fit and/or Predict

In [12]:
tokens = tokenizer.tokenize("ceci est un exemple !")

In [16]:
camembert.generate(tokens)

AttributeError: You tried to generate sequences with a model that does not have a LM Head.Please use another model class (e.g. `OpenAIGPTLMHeadModel`, `XLNetLMHeadModel`, `GPT2LMHeadModel`, `CTRLLMHeadModel`, `T5WithLMHeadModel`, `TransfoXLLMHeadModel`, `XLMWithLMHeadModel`, `BartForConditionalGeneration` )

# 4 Use pipeline directly

In [24]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer,AutoModelForQuestionAnswering

# Sentiment analysis pipeline
#pipeline('sentiment-analysis')

# Question answering pipeline, specifying the checkpoint identifier
#pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')

# Named entity recognition pipeline, passing in a specific model and tokenizer
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
pip=pipeline('ner', model=model, tokenizer=tokenizer)

In [22]:
pip.predict(["ceci est un exemple !"])

[{'word': '<s>', 'score': 0.5158082246780396, 'entity': 'LABEL_1'},
 {'word': '▁ceci', 'score': 0.5247928500175476, 'entity': 'LABEL_0'},
 {'word': '▁est', 'score': 0.5512601137161255, 'entity': 'LABEL_0'},
 {'word': '▁un', 'score': 0.5659816265106201, 'entity': 'LABEL_0'},
 {'word': '▁exemple', 'score': 0.5482860803604126, 'entity': 'LABEL_0'},
 {'word': '▁!', 'score': 0.5466126799583435, 'entity': 'LABEL_0'},
 {'word': '</s>', 'score': 0.5237824320793152, 'entity': 'LABEL_1'}]